In [16]:
from test_surrogate import  fit_surrogate_model
import plotly.graph_objects as go
import numpy as np
import torch

x_bounds = torch.tensor([[-np.pi], [np.pi]])
x = torch.tensor(
    [
        [-3 * np.pi / 4],
        [-np.pi / 4],
        [-np.pi / 8],
        [np.pi / 8],
        [np.pi / 4],
        [3 * np.pi / 4],
    ],
)
y = torch.cat([torch.sin(x), torch.cos(x)], dim=-1).squeeze()
y_1d = torch.sin(x)
x_eval = torch.tensor([[-np.pi / 2], [0], [np.pi / 2]])
expected_y_pred = torch.tensor(
    [[-5.6474e-01, -8.0338e-02], [1.1183e-16, 9.4965e-01], [5.6474e-01, -8.0338e-02]],
    dtype=torch.float64,
)
expected_y_pred_1d = torch.tensor(
    [[-5.6474e-01], [1.1183e-16], [5.6474e-01]], dtype=torch.float64
)
x_eval_fine = np.linspace(-np.pi, np.pi, 100)

model = fit_surrogate_model(x, x_bounds, y, device="cpu", dtype=torch.float64, model="joint", normalise=False)

figure = go.Figure(
    data=[
    go.Scatter(
        x=x.squeeze().detach().cpu().numpy(),
        y=y[:, 0].detach().cpu().numpy(),
        mode="markers",
        marker_symbol="x",
        marker_color="red",
        name="Observations 1",
    ),
    go.Scatter(
        x=x.squeeze().detach().cpu().numpy(),
        y=y[:, 1].detach().cpu().numpy(),
        mode="markers",
        marker_symbol="cross",
        marker_color="blue",
        name="Observations 2",
    ),
    go.Scatter(
        x=x_eval_fine,
        y=np.sin(x_eval_fine),
        mode="lines",
        line_color="red",
        line_dash="dash",    
        name="Truth 1"
    ),
    go.Scatter(
        x=x_eval_fine,
        y=np.cos(x_eval_fine),
        mode="lines",
        line_color="blue",
        line_dash="dash",
        name="Truth 2"
    ),
    go.Scatter(
        x=x_eval_fine,
        y=model.posterior(torch.tensor(x_eval_fine).unsqueeze(-1)).mean[:, 0].detach().cpu().numpy(),
        mode="lines",
        line_color="red",
        name="Posterior 1"
    ),
    go.Scatter(
        x=x_eval_fine,
        y=model.posterior(torch.tensor(x_eval_fine).unsqueeze(-1)).mean[:, 1].detach().cpu().numpy(),
        mode="lines",
        line_color="blue",
        name="Posterior 2"
    )
    ]
)
figure.show()


In [17]:
from jetto_mobo.acquisition import generate_trial_candidates, qNoisyExpectedHypervolumeImprovement

model = fit_surrogate_model(x, x_bounds, y, device="cpu", dtype=torch.float64, model="joint", normalise=False)

candidates = generate_trial_candidates(
        x,
        x_bounds,
        model,
        acquisition_function=qNoisyExpectedHypervolumeImprovement,
        batch_size=5,
        device="cuda",
        dtype=torch.float64,
        acqf_kwargs=dict(
            ref_point=torch.tensor([0.0, 0.0]),
            prune_baseline=True,
        ),
    )

for candidate in candidates:
    figure.add_vline(candidate.cpu().numpy()[0])
    
figure.show()